In [1]:
from pyspark.sql.session import SparkSession
from pyspark.sql.context import SQLContext
import requests
import csv

In [2]:
#API endpoint to get the information we required
url = "https://data.sfgov.org/resource/nuek-vuh3.csv"
r = requests.get(url)

if r.status_code == 200:
    with open("../data/fire_department_calls.csv", "w") as csvfile:
        csv_writer = csv.writer(csvfile)
        headers = r.text.splitlines()[0].split(',')
        csv_writer.writerow(headers)

        for row in r.text.splitlines()[1:]:  # Skip the header row
            data_row = row.split(',')
            csv_writer.writerow(data_row)

In [3]:
#spark session
spark = SparkSession.builder.appName("fire_calls").getOrCreate()
#sql context
sqlcontext = SQLContext(spark)

c:\Users\juanf\anaconda3\envs\dataenv\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
#reading a csv file.
sf_fire_calls_df = spark.read.csv("../data/fire_department_calls.csv", header=True, inferSchema=True)


In [5]:
sf_fire_calls_df.printSchema()

root
 |-- """call_number""": string (nullable = true)
 |-- """unit_id""": string (nullable = true)
 |-- """incident_number""": string (nullable = true)
 |-- """call_type""": string (nullable = true)
 |-- """call_date""": string (nullable = true)
 |-- """watch_date""": string (nullable = true)
 |-- """received_dttm""": string (nullable = true)
 |-- """entry_dttm""": string (nullable = true)
 |-- """dispatch_dttm""": string (nullable = true)
 |-- """response_dttm""": string (nullable = true)
 |-- """on_scene_dttm""": string (nullable = true)
 |-- """transport_dttm""": string (nullable = true)
 |-- """hospital_dttm""": string (nullable = true)
 |-- """call_final_disposition""": string (nullable = true)
 |-- """available_dttm""": string (nullable = true)
 |-- """address""": string (nullable = true)
 |-- """city""": string (nullable = true)
 |-- """zipcode_of_incident""": string (nullable = true)
 |-- """battalion""": string (nullable = true)
 |-- """station_area""": string (nullable = true

In [6]:
#sf_fire_calls_df.show(10)

We have to create a global-temporary view, which is similar to a table. I can run SQL expression on this view

In [7]:
sf_fire_calls_df.createOrReplaceTempView("fire_service_calls_view")

In [8]:
query_1 = """
        SELECT * FROM fire_service_calls_view
"""

In [9]:
#spark.sql(query_1).show()